# Object removal

This app allows you to remove objects from images.

**This application is hosted in [Ploomber Cloud](https://ploomber.io/).**

First, upload an image:

In [1]:
from ipywidgets import FileUpload, Image, ColorPicker, IntSlider, link, AppLayout, HBox, Output, Button, VBox
from ipycanvas import Canvas, hold_canvas, MultiCanvas
import PIL
from IPython import display
import time

In [2]:
drawing = False
position = None
shape = []


def on_mouse_down(x, y):
    global drawing
    global position
    global shape

    drawing = True
    position = (x, y)
    shape = [position]


def on_mouse_move(x, y):
    global drawing
    global position
    global shape

    if not drawing:
        return

    with hold_canvas():
        canvas.stroke_line(position[0], position[1], x, y)

        position = (x, y)

    shape.append(position)


def on_mouse_up(x, y):
    global drawing
    global position
    global shape

    drawing = False

    with hold_canvas():
        canvas.stroke_line(position[0], position[1], x, y)
        canvas.fill_polygon(shape)

    shape = []


multi = MultiCanvas(2, width=10, height=10, sync_image_data=True)
background, canvas = multi
canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

In [3]:
import io

upload = FileUpload(
    accept='image/*',
    multiple=False
)

output = Output()


img_pil = None


def on_file_upload(change):
    uploaded_file = change['new'][0]
    content = uploaded_file['content']

    global img_pil
    img_pil = PIL.Image.open(io.BytesIO(content.tobytes()))
    multi.width, multi.height = img_pil.width, img_pil.height
    
    image = Image(value=content)
        
    background.fill_style = "#000000"
    background.fill_rect(0, 0, 1000, 1000)
    background.draw_image(image, x=0, y=0)
    canvas.stroke_style = "#000000"
    canvas.sync_image_data = True

    
upload.observe(on_file_upload, names='value')
upload

FileUpload(value=(), accept='image/*', description='Upload')

The image will appear here, draw the silhouette from the object you want to remove:

In [4]:
multi

MultiCanvas(height=10, sync_image_data=True, width=10)

When done, click on "Remove object":

In [5]:
import ipyspin

s = ipyspin.Spinner()

s.layout.height = '50px'
s.layout.width = '50px'

s.lines = 10  # The number of lines to draw
s.length = 38/4  # The length of each line
s.width = 17/4  # The line thickness
s.radius = 40/4  # The radius of the inner circle
s.scale = 1  # Scales overall size of the spinner
s.corners = 1  # Corner roundness (0..1)
s.speed = 0.5  # Rounds per second
s.rotate = 0  # The rotation offset
s.animation = 'spinner-line-fade-quick'  # The CSS animation name for the lines
s.direction = 1  # 1: clockwise, -1: counterclockwise
s.color = '#000000'  # CSS color or array of colors
s.fade_color = 'transparent'  # CSS color or array of colors
s.top = '50%'  # Top position relative to parent
s.left = '50%'  # Left position relative to parent
s.shadow = '0 0 1px transparent'  # Box-shadow for the lines
s.z_index = 2000000000  # The z-index (defaults to 2e9)
s.class_name = 'spinner'  # The CSS class to assign to the spinner
s.position = 'absolute'  # Element positioning

In [6]:
import replicate
import requests

def run_model():
    url = None

    try:
        url =  replicate.run(
    "zylim0702/remove-object:0e3a841c913f597c1e4c321560aa69e2bc1f15c65f8c366caafc379240efd8ba",
    input={"image": open("tmp_orig.jpg", "rb"),
           "mask": open("tmp.jpg", "rb")
          }

    
    )
    except Exception as e:
        pass

    if not url:
        raise RuntimeError("An error happened when running the model, please try again")

    return requests.get(url).content

In [8]:
button = Button(description="Remove object")

def on_button_clicked(button):
    mask = PIL.Image.fromarray(canvas.get_image_data())
    mask_ = PIL.Image.new("RGBA", mask.size, "WHITE")
    mask_.paste(mask, (0, 0), mask)           
    mask_ = mask_.convert('RGB')
    mask_final = PIL.ImageOps.invert(mask_)
    mask_final.save("tmp.jpg")

    global img_pil
    img_pil.save("tmp_orig.jpg")

    with output:
        display.clear_output()
        display.display(s)
        img_data = run_model()
        display.clear_output()
        display.display(display.Image(data=img_data))

button.on_click(on_button_clicked)

VBox([button])

You can try it multiple times to remove objects iteratively.

(right click to save the image)

In [9]:
output

Output()